In [ ]:
!pip install peft -q
!pip install -i https://pypi.org/simple/ bitsandbytes -q
!pip install accelerate -q
!pip install datasets -q
!pip install zstandard -q
!pip install jsonlines -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 62.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch

MODEL_NAME = "IlyaGusev/saiga_7b_lora"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты оцениваешь правильный ли ответ на вопрос с учетом контекста, да если ответ правильный и нет если ответ неправильный."

class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        start_token_id=1,
        bot_token_id=9225
    ):
        self.message_template = message_template
        self.start_token_id = start_token_id
        self.bot_token_id = bot_token_id
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def get_start_token_id(self):
        return self.start_token_id

    def get_bot_token_id(self):
        return self.bot_token_id

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += tokenizer.decode([self.start_token_id, self.bot_token_id])
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16,
    is_trainable = True,
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/419 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


generation_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}



In [ ]:
import pandas as pd


In [ ]:
df = pd.read_csv("train.csv")

In [ ]:
print(df.loc[11].answer)

в 1991 году


In [ ]:

def answer(summary, question, answer, is_hallucination):
    inputs = f"""{summary}
             Задан вопрос: {question}
             Ответ: {answer}
             Правильный ли ответ?
             """

    conversation = Conversation()
    conversation.add_user_message(inputs)
    prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)
    print(inputs)
    print(output)
    print(output.lower().split(" "))
    print("Predicted: ", 0 if "да" in output.lower().replace(",", "").split(" ") else 1)
    print("True: ", is_hallucination)
    print()
    print("==============================")
    print()
    return  1 if "да" in output.lower() else 0

In [ ]:
import pandas as pd
import json

def dataset_to_json(dataset, filename):
    json_objects = []

    for index, row in dataset.iterrows():
        system_message = "Ты — Сайга, русскоязычный автоматический ассистент. Ты оцениваешь правильный ли ответ на вопрос с учетом контекста, да если ответ правильный и нет если ответ неправильный."
        user_message = f"{row['summary']} \nЗадан вопрос: {row['question']} \nОтвет: {row['answer']} \nПравильный ли ответ? \n"
        bot_message = "Да" if row['is_hallucination'] == 0 else "Нет"

        json_object = {
            "system": system_message,
            "user": user_message,
            "bot": bot_message
        }

        json_objects.append(json_object)
        with open(filename, 'a', encoding='utf-8') as file:
          file.write(json.dumps(json_object, ensure_ascii=False) + "\n")

    return json_objects


In [ ]:
train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state=200)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))
dataset_to_json(train_dataset, "train.json")
dataset_to_json(test_dataset, "test.json")


FULL Dataset: (1050, 5)
TRAIN Dataset: (840, 5)
TEST Dataset: (210, 5)


[{'system': 'Ты — Сайга, русскоязычный автоматический ассистент. Ты оцениваешь правильный ли ответ на вопрос с учетом контекста, да если ответ правильный и нет если ответ неправильный.',
  'user': 'Херманус Питер (Дик) Логгере (нидерл. Hermanus Pieter (Dick) Loggere, 6 мая 1921, Амстердам, Нидерланды — 30 декабря 2014, Хилверсюм, Нидерланды) — нидерландский хоккеист (хоккей на траве), полузащитник. Бронзовый призёр летних Олимпийских игр 1948 года. \nЗадан вопрос: В каком городе проходил чемпионат мира по хоккею с шайбой в 1936 году? \nОтвет: В Хилверсюме. \nПравильный ли ответ? \n',
  'bot': 'Нет'},
 {'system': 'Ты — Сайга, русскоязычный автоматический ассистент. Ты оцениваешь правильный ли ответ на вопрос с учетом контекста, да если ответ правильный и нет если ответ неправильный.',
  'user': 'Чёрная вдова (лат. Latrodectus mactans) — вид пауков, распространённый в Северной и Южной Америке. Опасен для человека. \nЗадан вопрос: Для кого опасны пауки-бокоходы? \nОтвет: Для рыб. \nПравил

In [ ]:
from datasets import load_dataset
data = load_dataset(
    "json",
    data_files={
                'train' : '/content/train.json' ,
                'validation' : '/content/test.json'
    }
)
data

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['system', 'user', 'bot'],
        num_rows: 840
    })
    validation: Dataset({
        features: ['system', 'user', 'bot'],
        num_rows: 210
    })
})

In [ ]:
CUTOFF_LEN = 3584

def generate_prompt(data_point):
    promt = f"""<s>system
{data_point['system']}</s><s>user
{data_point['user']}</s><s>bot
{data_point['bot']}</s>"""
    #     print(promt)
    return promt


def tokenize (prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id and len(result["input_ids"]) < CUTOFF_LEN
        and add_eos_token
    ):

        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)



    result["labels"] = result["input_ids"].copy()

    return result

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenize(full_prompt)
#     print(tokenized_full_prompt)
    return tokenized_full_prompt

In [ ]:
train_data = (
    data["train"].map(generate_and_tokenize_prompt)
)

val_data = (
    data["validation"].map(generate_and_tokenize_prompt)
)

Map:   0%|          | 0/840 [00:00<?, ? examples/s]

Map:   0%|          | 0/210 [00:00<?, ? examples/s]

In [ ]:
import torch
import pandas as pd
from peft import PeftModel, PeftConfig
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForTokenClassification, AutoConfig, GenerationConfig
from transformers import Trainer, TrainingArguments, logging, TrainerCallback, TrainerState, TrainerControl, BitsAndBytesConfig
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
import torch.nn.functional as F
from datasets import load_dataset
import time
from typing import Any, List, Mapping, Optional
import transformers
import os
from pathlib import Path
BATCH_SIZE = 2
MICRO_BATCH_SIZE = 1
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_STEPS = 100
OUTPUT_DIR = "/content/drive/MyDrive/Models/saiga_hallucinations"

training_arguments = transformers.TrainingArguments(
            per_device_train_batch_size=MICRO_BATCH_SIZE,
            gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
#             warmup_steps=200,
            max_steps=TRAIN_STEPS,
            learning_rate=LEARNING_RATE,
            fp16=True,
            logging_steps=10,
            optim="adamw_torch",
            evaluation_strategy="steps",
            save_strategy="steps",
            eval_steps=10,
            save_steps=10,
            output_dir=OUTPUT_DIR,
            save_total_limit=10,
            load_best_model_at_end=True,
            report_to=None,
            overwrite_output_dir=True, # Overwrite the content of the output dir
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:

data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=training_arguments,
    data_collator=data_collator
)
model = torch.compile(model)
trainer.train()


max_steps is given, it will override any value given in num_train_epochs
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Step,Training Loss,Validation Loss
10,1.244900,1.042627
20,0.926100,1.002404
30,0.841600,0.997268
40,0.834900,0.992478
50,0.823400,0.988511
60,0.906000,0.986780
70,0.815400,0.982550
80,0.818600,0.979999
90,0.805900,0.979227
100,0.859500,0.978783


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

In [ ]:
df["answer"] =  df.apply(lambda x: answer(x.summary, x.question, x.answer, x.is_hallucination), axis=1)

Выходные данные были обрезаны до нескольких последних строк (5000).
             Ответ: Формо
             Правильный ли ответ?
             
Да
['да']
Predicted:  0
True:  0


«Валери Соланас умерла за твои грехи: Тварь» (англ. «Valerie Solanas Died for Your Sins: Scumbag») — седьмой эпизод седьмого сезона американского телесериала-антологии «Американская история ужасов»., премьера которого состоялась 17 октября 2017 года на телеканале FX. Режиссёром стала Рейчел Голдберг, а сценаристом Кристал Лью.
             Задан вопрос: Какой по счету был эпизод сериала?
             Ответ: Восьмой
             Правильный ли ответ?
             
Нет
['нет']
Predicted:  1
True:  1


Нана Анима Виафе-Акентен (англ. Nana Anima Wiafe-Akenten) — ганская лингвист и ведущая на телевидении и радио. Заведующая кафедрой Акан-Нзема Колледжа языкового образования, в кампусе Аджумако Университета образования, Виннеба в Гане. Первый человек, получивший докторскую степень защитив диссертацию на языке тви.
    

In [ ]:
from sklearn import metrics
accuracy = metrics.accuracy_score(df["is_hallucination"],df["answer"])
f1_score_micro = metrics.f1_score(df["is_hallucination"],df["answer"], average='micro')
f1_score_macro = metrics.f1_score(df["is_hallucination"],df["answer"], average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

Accuracy Score = 0.35523809523809524
F1 Score (Micro) = 0.3552380952380953
F1 Score (Macro) = 0.34305314629347416


In [ ]:
model.save_pretrained(OUTPUT_DIR)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
